In [33]:
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import evaluate

In [38]:

test_dataset_names = ["ISEAR", "DR", "Dreaddit", "SDCNL", "DepSeverity"]
data_path = "/home/elicer/Tiny-Mental-LLM/Mental_Emotion_Dataset/final_merged_dataset/test"
if(data_path == ""):
  raise Exception("Path for dataset not found")

token_path = "Qwen/Qwen2-0.5B"
model_path = "/home/elicer/Tiny-Mental-LLM/train_and_evaluate/Output/csidibn1-Qwen-Qwen2-0.5B/checkpoint-3880"

In [35]:
def _get_text_label_from_path(path):
    # Assuming that the dataset file is csv and contains `text` & `label` columns
    texts = []
    labels = []
    labels_set = set()
    with open(path, "r", encoding = "utf-8") as test_file:
        reader = csv.reader(test_file)
        text_id = 0
        label_id = 1
        for id, _row in enumerate(reader):
            if(id == 0 or len(_row) < 2):
                continue
            else:
                texts.append(_row[text_id])
                labels.append(label2id[_row[label_id]])
                labels_set.add(_row[label_id])
        test_file.close()
    assert (len(texts) == len(labels)), "Number of text {} is different from number of labels {}".format(len(texts), len(labels))
    return texts[:10], labels[:10]


In [36]:
def compute_metrics(preds, labels):
    accuracy_metric = evaluate.load("f1", trust_remote_code = True)
    precision_metric = evaluate.load("precision", trust_remote_code = True)
    recall_metric = evaluate.load("recall", trust_remote_code = True)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels, average = 'weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average = 'weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average = 'weighted')
    return {**accuracy, **precision, **recall}

In [39]:
from evaluate import evaluator
from evaluate import load, combine
import evaluate
import csv
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import evaluate

token = "" #Huggingface token to access the model
# Load model directly
tokenizer = AutoTokenizer.from_pretrained(token_path, token = token, model_max_length = 2048, truncation_side = "left")
new_model = AutoModelForSequenceClassification.from_pretrained(f"{model_path}") # Check the config.json for the proper name

label2id = model_config.label2id
print("Label2id", label2id)

new_model.eval()




for test_dataset_name in test_dataset_names:
    # _test_texts, _test_labels = _get_text_label_from_path(f"{data_path}/{test_dataset_name}.csv")
    # _test_encodings = tokenizer(_test_texts , truncation = True)
    # _test_dataset = CustomDataset(_test_labels, _test_encodings)

    dataset = Dataset.from_csv(f"{data_path}/{test_dataset_name}.csv")
    dataset = dataset.map(lambda example: {"label": label2id[example["label"]], 'input_ids': tokenizer(example['text']).input_ids})

    task_evaluator = evaluator("text-classification")


    # 1. Pass a model name or path
    train_args = TrainingArguments(
        output_dir = "/home/elicer/Tiny-Mental-LLM/train_and_evaluate/results",
        do_train = False,
        do_eval = True,
        per_device_eval_batch_size = 1,
    )

    trainer = Trainer(
        model=new_model,
        args=train_args
    )

    results = trainer.predict(test_dataset = dataset)
    
    preds = results.predictions
    labels = results.label_ids

    preds = torch.from_numpy(preds)
    preds = torch.argmax(preds, dim = -1)
    labels = torch.from_numpy(labels)

    print(f"For dataset {test_dataset_name}, the results are: {compute_metrics(preds, labels)}")


Label2id {'anger': 0, 'depression': 1, 'disgust': 2, 'fear': 3, 'guilt': 4, 'joy': 5, 'mild': 6, 'minimum': 7, 'moderate': 8, 'no': 9, 'sadness': 10, 'severe': 11, 'shame': 12, 'suicidal': 13, 'yes': 14}


KeyboardInterrupt: 